<a href="https://colab.research.google.com/github/computas/ml-intro/blob/master/101-data-som-virkelighet/OppgaveDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problemstilling

Tenkt deg at datoen er 29. april 2020. Mange byer og stater i verden begynner å se de aller første tilfellene av coronaviruset, noen steder har pandemien kommet ganske langt allerede. New York State er en av statene der pandemien har kommet langt, og de er også gode til å samle inn data. Din oppgave er å lage en modell for å **predikere hvilke bydeler som er mest utsatt for å få mange dødsfall fra viruset**. Modellen skal andvendes i andre storbyer i verden i det de oppdager sine første koronatilfeller. Meningen er at man skal gå inn og teste mange, samt gjøre andre forebyggende tiltak i bydeler der man predikerer at antall dødsfall kommer til å bli høyt.

For å evaluere modellen bruker vi [leave-one-out cross validation](https://link.springer.com/referenceworkentry/10.1007%2F978-0-387-30164-8_469), og vi evaluerer med RMS. Det er skrevet en metode som gjør dette.

### Forklaring til dataene

Alle dataene er oppdatert 28. april 2020.

|Variabel | Forklaring
|--------- | -------------
|deaths | registrerte dødsfall
|cumulative_positives | totalt antall som har testet positivt
|cumulative_tests | totalt antall som er testet
|medianFamily  | median inntekt for en familiy
|density  | antall personer som bor per km2
|population | befolkning
|smoking   | prosentandel som røyker
|infected  | andel av befolkingen som har blitt syke
|county    | staten New York er delt inn i counties
|arealand      | arealet til county

### Datakilder
* Antall døde i sentrum av New York per county
https://www1.nyc.gov/site/doh/covid/covid-19-data-archive.page
* Antall døde i hvert county, men flere i sentrum blir telt sammen, dermed ble det kombinert med forrige
https://github.com/nytimes/covid-19-data (Data from The New York Times, based on reports from state and local health agencies.)
* Data om antall testede
https://data.ny.gov/
* Andel som røyker
https://health.data.ny.gov/Health/Community-Health-Indicator-Reports-CHIRS-Latest-Da/54ci-sdfi
* Populasjon og areal av counties
https://www.wolframalpha.com/

# Oppgaven

Vi har skrevet en notebook som tar deg gjennom å utforske dataene, samt lage en modell. 
Ha hele tiden oppgavens problemstilling i bakhode mens du jobber deg gjennom denne notebook

1. Gjør utforskende dataanalyse. Det skal ikke være noe galt med dataene, vi har allerede gått igjennom følgende sjekkliste for å se at alt stemmer: https://docs.google.com/document/d/1km5mnhdYrAO3_2u0B06YSYPQLykuTQmujAuwjQjfAeA/edit?usp=gmail&ts=5e97143a. Det eneste er at vi er på grensen til å ha litt for lite data til å lage en god modell. En slik sjekkliste er veldig nyttig, men noen ganger ikke nok. Det er også viktig å tenke seg godt om når man utforsker dataene og tester ut modeller.
Husk spesielt å:
* Plotte distribusjonene til hver av variablene
* Lag scatterplot for å se på korrelasjon mellom variablene

2. Prøv å sett opp en enkel modell, test linear regresjon og random forest til å begynne med. 
For å evaluere modellen bruker vi leave-one-out cross validation, og vi evaluerer med [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Det er skrevet en metode som gjør dette. 








In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import sklearn
from sklearn import preprocessing
import sklearn.feature_selection
from sklearn.model_selection import (train_test_split,
                                     cross_val_score,
                                     cross_val_predict,
                                     cross_validate)
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor  

In [2]:
#@title Definer `corr_heatmap` funksjonen (du må kjøre denne cellen)
def corr_heatmap(df, figsize=(10,10)):
    _, axs = plt.subplots(figsize=figsize)
    ax = sns.heatmap(df, vmin=-1, vmax=1, cmap="BrBG", linewidths=0.5, annot=True, ax=axs)
    ax.set_title('Correlation matrix')
    return ax

In [3]:
#@title Definer `makeHistogram` funksjonen (du må kjøre denne cellen)
#Hjelpefunksjon for histogrammer
def makeHistogram(data,variable,binwidth,upper_lim=None,lower_lim=None):
    if upper_lim==None:
        upper_lim=data[variable].max()
    if lower_lim==None:
        lower_lim=data[variable].min()
    bins=np.arange(lower_lim, upper_lim + binwidth, binwidth)
    plt.hist(data[variable],bins=bins,range=[data[variable].min(),upper_lim]) 
    plt.ylabel("frequency")
    plt.xlabel(variable)

In [4]:
#@title Definer `makeScatterPlot` funksjonen (du må kjøre denne cellen)
def makeScatterPlot(data,para1,para2,logx=False,logy=False):
    plt.plot(data[para1],data[para2],'*')
    if logx:
        plt.xscale("log")
    if logy:    
        plt.yscale('log')
    plt.xlabel(para1)
    plt.ylabel(para2)

Last ned data fra git (bare hvis du kjører fra colab)


In [ ]:
# Hent data
!wget -c https://raw.githubusercontent.com/computas/ml-intro/master/101-data-som-virkelighet/newYorkData.csv

# Importer dataene og få en første oversikt

In [ ]:
data = pd.read_csv("newYorkData.csv")
data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

# Utforsk dataene

Utforsk dataene:
- Hvilke variable ser ut til å påvirke antall dødsfall?
- Dersom dere skulle lage en modell, hvordan ville dere satt den opp?
- Hvilke variable ville dere brukt? 

Dere har fått to hjelpefunksjoner for å lage histogrammer og scatterplot av dataene.

Dersom du er usikker på hva et histogram eller et scatterplot er kan det være nyttig å lese dette: 

Histogrammer: https://statistics.laerd.com/statistical-guides/understanding-histograms.php

Scatterplot: https://chartio.com/learn/charts/what-is-a-scatter-plot/
Eller du kan spørre en av medhjelperene

Vi kommer også til å undersøke scatterplots og korrelasjoner. Er du litt usikker på disse begrepene, kan du lese følgende:
https://www.westga.edu/academics/research/vrc/assets/docs/scatterplots_and_correlation_notes.pdf

Hvis `r` er korrelasjonsverdien, vi kan klassifisere styrken av korrelasjonen via følgende tabell:

Absolute Value of r | Strength of Relationship
--- | ---
r < 0.3 | None or very weak
0.3 < r <0.5 | Weak
0.5 < r < 0.7 | Moderate
r > 0.7 | Strong

Om du vil trene på korrelasjon og scatterplots, kan du ta noen runder med følgende spill:
http://guessthecorrelation.com/

Vi begynner nmed å lage en correlation heat map for å få en oversikt over hvor det kan være korrelasjoner. 

In [ ]:
corr_heatmap(data.corr())

# Sjekk distribusjonen av variable og korrelasjonen mellom variablene


Plot histogrammene for de variablene du mener er mest interessante

In [ ]:
makeHistogram(data,"cumulative_positives",lower_lim=0,upper_lim=None,binwidth=1000)

Plot dine egne histogrammer

In [ ]:
makeHistogram(data,....)

Vi ser det er en del sterke korrelasjoner. Lag scatterplot for de du mener er interessante, prøv også å bruke log skala. Spesielt ser det ut til at population er korrelert med dødsfall. For hvert scatterplot du lager, tenk igjennom hvorfor det ser ut som det gjør. 

In [ ]:
makeScatterPlot(data,"medianFamily","deaths",logx=False, logy=False)

Plot dine egne scatterplot

In [ ]:
makeScatterPlot(data,...)

# Diskuter
Det ser ut til at både `cumulative_tests`, `cumulative_ positives` er gode prediktorer. Men allikevel er det ikke sikkert at de bør brukes når man lager en modell. 

Gå tilbake og les problemstillingen en gang til, diskuter om det er variable vi ikke kan bruke. (hint: har vi tilgjengelig alle variabler når vi skal predikere?)

<details> 
  <summary> Trykk på pilen først etter at dere har diskutert </summary>
  Modellen skal brukes i det de første tilfellene dukker opp i en by. På det tidspunktet vet man ikke noe om hvor mange som er smittet, hvor mange som er testet eller hvor mange som har testet positivt. Man ønsker å bruke modellen til å påvirke disse variablene, dermed kan de ikke være en del av modellen
</details>

# Prøv dere frem med noen enkle modeller og diskuter. 

Basert på konklusjonen av diskusjonen i forrige oppgave fokuserer vi på `population`, `medianFamily`, `smoking` og `density`. 


Vi tester en enkel random forest modell. Random forest er ikke avhenige av lineære sammenhenger. Vi har skrevet en enkel funksjon for dere som gjøre leave-one-out cross validering av modellen. Test modellen med forskjellig kombinasjon av uavhenige variable. Sjekk feature importance. Hvilke features ser ut til å være viktige:

In [24]:
#@title Definer `train_model` Funksjonen trener modellen og returnerer prediksjonene, rms, modellen og feature importance.
def train_model(data, columns):
    regressor = RandomForestRegressor(random_state=4, max_depth=3)

    def predict(train, test):
        test = test[columns]
        X = train[columns]
        y = train["deaths"]
        regressor.fit(X, y)
        prediction = regressor.predict(test)
        return prediction

    predictions = []
    result = data[columns]
    result["deaths"] = data["deaths"]
    for i in range(len(data)):
        predicted = predict(data.drop([i]), data.iloc[[i]])
        predictions.append(predicted[0])
    result["predictions"] = predictions
    result["diff"] = result["predictions"] - result["deaths"]
    error = np.sqrt(mean_squared_error(result["deaths"], result["predictions"]))
    model = regressor.fit(data[columns], data["deaths"])
    # Add shap importances, example: https://www.kaggle.com/wrosinski/shap-feature-importance-with-feature-engineering
    importance = model.feature_importances_
    return result, error, model, importance

In [41]:
#@title Definer `print_evaluation`. Funksjonen skriver ut RMS og feature importance
def print_evaluation(rms:float,feature_importance:list, features:list):
  print(f"RMS: {rms}")
  print("Feature importance:")
  for items in list(zip(features,feature_importance)):
    print(f"{items[0]}: \t {items[1]}")
  return

Prøv og bruk et subset av uanvhenige variable

In [25]:
# Define columns
columns=["population","density","medianFamily","smoking"]

# Train your model
result,rms,model,importance = train_model(data,columns)

In [ ]:
print_evaluation(rms,importance,columns)

<details>
 <summary> Trykk på pilen først etter at dere har prøvd dere frem litt </summary>
Egentlig så ser det ut som om det er best å bruke kun en uavhenige variabel. Dette gir mening, siden det er kun det vi har nok data til. Det ser ut til at "population" er den viktigste variablen
</details>

# La oss ta en tenkepause og diskutere litt igjen

Det ser ut til at den viktigste uavhenige variablen er `population`. 

Tenk igjennom hvorfor. 

- Hvilken verdi gir det egentlig for å ha en modell som predikerer antall dødsfall som funksjon av antall innbyggere?
- Dersom det ikke var noen forskjell på bydelene, hva ville sammenhengen mellom population og deaths da vært? 

Som nevnt i oppgaven så tenker vi oss nå at en annen by skal andvende modellen, men vil modellen slik den er nå gi verdi for den nye byen? Kan vi teste det på noen måte? Diskuter

<details> 
  <summary> Trykk på pilen først etter at dere har diskutert </summary>
    Det er ganske logisk at det er en sammenheng mellom antall folk i en bydel og hvor mange som dør av korona. Dersom alle bydelene har samme risiko, det vil si samme prosentandel som dør, så ville man sett en lineær sammenheng.
    Altså ville man sett en sterk korrelasjon mellom population og deaths selv om alle bydelene var like utsatt. Dersom modellen skal være overførbar til en annen by, burde modellen fungere selv om man valgte å dele inn bydelene på en annen måte. Vi kan gjøre en test ved å slå sammen alle bydelene med mindre enn 70000 inbyggere til en ny bydel, og se hva den modellen predikerer.
    </details>

# Gjør en county sammenslåing (kommunesammenslåing :))

Vi trener modellen med kun population som uavhenige variable

In [ ]:
columns = ["population"]
result,rms,model,importance = train_model(data,columns)
print_evaluation(rms,importance,columns)

Vi slår sammen alle counties som har mindre enn 70000 inbyggere til en ny county

In [29]:
# Hent alle datapunkter med mindre enn 70000 innbyggere
small = data[data["population"]<70000]
newCounty = pd.DataFrame()
newCounty = newCounty.append(pd.Series(),ignore_index=True)

# Beregn deaths, density og population for den nye datapunktet 
newCounty["deaths"] = small["deaths"].sum()
newCounty["density"] = small["population"].sum()/small["arealand"].sum()
newCounty["population"] = small["population"].sum()

In [ ]:
newCounty

In [ ]:
# Hva sier modellen?
prediction = model.predict(newCounty[columns])[0]
real = newCounty['deaths'][0]
print(f"Modellen predikerer {prediction} dødsfall, men i virkeligheten har vi bare {real} dødsfall")

Dette gikk jo veldig feil. I det nye county så predikerer vi altfor mange dødfall, men det reelle tallet var bare 50. Diskuter hva som skjedde her. Vil en modell som kun bruker density fungere bedre dersom vi slår sammen county. Hvorfor er det heller ikke ideelt å kun bruke density

<details> 
  <summary> Trykk på pilen først etter at dere har diskutert </summary>
Egentlig så bør vi ikke ha med estimere antall dødfall. Fordi det vi egentlig er ute etter er å estimere hvilke bydeler som er mer utsatt for korona enn andre, det vil si hvilke bydeler som har en høyest dødsrate (antall døde per inbygger). Dermed bør vi heller lage en modell der vi predikerer dødsraten. For å evaluere modellen slik vi ble bedt om av oppdragsgiver, estimerer vi dødraten i hver bydel før vi ganger vi med population for å estimere antall som døde. Det ville vært det samme som å 
    </details>

In [36]:
#@title Definer funksjonen `makePredictionsUsingDeathRate` (du må kjøre denne cellen)
def makePredictionsUsingDeathRate(data, columns, use_weight=False):
    regressor = RandomForestRegressor(
        n_estimators=200, random_state=42, bootstrap=True, max_depth=3
    )
    # regressor = DecisionTreeRegressor(max_depth=10)
    def predict(train, test):
        population = test.population.values[0]        
        test = test[columns]
        X = train[columns]
        y = train["deathRate"]
        if use_weight:
          train_population = train["population"]
          regressor.fit(X, y, sample_weight=train_population)
        else:
          regressor.fit(X, y)
        prediction = regressor.predict(test)
        return prediction[0] * population

    predictions = []
    result = data[columns]
    result["deaths"] = data["deaths"]
    for i in range(len(data)):
        predicted = predict(data.drop([i]), data.iloc[[i]])
        predictions.append(predicted)
    result["predictions"] = predictions
    result["diff"] = result["predictions"] - result["deaths"]
    error = np.sqrt(mean_squared_error(result["deaths"], result["predictions"]))
    model = regressor.fit(
        data[columns], data["deathRate"], sample_weight=data["population"]
    )
    importance = model.feature_importances_
    return result, error, model, importance

In [37]:
data["deathRate"] = data["deaths"]/data["population"]

In [38]:
columns=["density"]

In [51]:
result,rms,model,importance = makePredictionsUsingDeathRate(data,columns)

In [ ]:
print_evaluation(rms,importance,columns)

In [ ]:
#Bruker modellen igjen for å predikere for det nye sammenslåtte county, og nå går det mye bedre
predicted_death_rate = model.predict(newCounty[columns])[0]
predicted_deaths = predicted_death_rate*newCounty["population"][0]
real_deaths = newCounty['deaths'][0]
print(f"Modellen predikerer {predicted_deaths} dødsfall, og i virkeligheten har vi {real_deaths} dødsfall")

Hva skjer med feature importance dersom vi bruker både både "population" og "density"?



Vi ser at vi fikk en noe bedre modell bare ved å faktisk predikere riktig variabel. Men nå tenker vi på hver enkel bydel som et målepunkt. Er det riktig å gjøre? Har det noe å si om en bydel har flere inbyggere enn en annen.

<details> 
  <summary> Trykk på pilen først etter at dere har diskutert </summary>
    I en bydel med stor befolkning har vi flere målepunkter, vi kan tenke på et dødsfall som et målepunkt. Det vil si at vi er mer sikre på dødsraten i en bydel med mange innbyggere. Dermed kan det være lurt å vekte dataene våre med population. Det kan gjøres ved å legge til parameter use_weight i funksjonen.
    <code> makePredictionsUsingDeathRate(data,columns, use_weight=True) <code>
    </details>

In [60]:
# Try to use the hint above


In [ ]:
# Se på koden her under om du ikke finner frem.

In [ ]:
#@title Hint: Using density and population. Weighthing by population.
columns=["density","population"]
result,rms,model,importance = makePredictionsUsingDeathRate(data,columns, use_weight=True)
print_evaluation(rms,importance,columns)

In [ ]:
#@title Hint: Using density. Weighthing by population.
columns=["density"]
result,rms,model,importance = makePredictionsUsingDeathRate(data,columns, use_weight=True)
print_evaluation(rms,importance,columns)

# Hva har vi egentlig endt opp med?

Vi endte opp med en modell som bruker kun density til å predikere death rate. Siden vi kun har en uavhenige variable kan vi plotte opp modellen.


In [ ]:
density = np.arange(data["density"].min(),data["density"].max())
predicted = model.predict(density.reshape(-1,1))

In [ ]:
plt.plot(density,predicted)
plt.plot(data["density"],data["deathRate"],'*')
plt.xlabel("density")
plt.ylabel("death rate")

Vi ser at vi egentlig bare har gjort noe som ligner på en enkel kurvetilpassing. Konklusjonen av alt vi har gjort er at tettboddhet gjør en bydel utsastt for korona. Det ser også ut som om vi har en slags terskel, når man kommer over en tetthet på rundt 1000 personer per km^2 så begynner det å bli veldig farlig. Men alt dette kunne vi egentlig ha sett kun ved å plotte density mot deathRate.

Ga alt vi gjorde med maskinlæring egentlig noe verdi? Diskuter

<details> 
  <summary> Trykk på pilen først etter at dere har diskutert </summary>
  
  I prosessen med å lage modellen så lært vi en del nyttig. Selv om konklusjonen vi kom frem til kunne man funnet ved kun å plotte deathRate mot density, så var veien dit lærerik. 
    
- Det viktigste vi fant ut var at de andre variablene ikke hadde så mye å si. For eksempel hadde det lite å si hvor mange som røyket i bydelen, og hvor rik bydelen var. 

- Dersom man ikke hadde gått igjennom prosessen med å lage modellen, så hadde man fremdeles lurt på om disse variablene hadde noe å si. Men ved å lage modellen så ble vi mye sikrere på at tettboddhet er veldig viktig, mye viktigere enn de andre faktorene vi hadde informasjon om. 

- Vi hadde også så lite data at man bare kunne bruke en uavhenige variable for å lage en modell. Gjennom denne prosessen så fant vi ut hvilken variabel som var den viktigste, og vi fant ut at dersom man skal lage en bedre modell trenger man mere data, man må kanskje aggregere data fra flere andre byer.

- Det er vår plikt å ikke være "teknologi-blinde" og fokusere bare på å bygge noe fordi vi kan, men vi må også validere at ting gir mening, og evt lære opp kunden vår i det de egentlig trenger. Kunden hadde bedt oss om å predikere dødsfall, og dette villedet oss litt på starten, men vi fant ut riktig tilnærming etter hvert.

    </details>

# Konklusjoner

 - Hva har vi lært?
 